In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms
from torchvision.utils import save_image
import time
import matplotlib.pyplot as plt
import numpy as np
from torchviz import make_dot
from torchinfo import summary
import torch.nn.functional as F  # Import functional for interpolation

# Define constants
IMG_SIZE = 512  # Updated to 512x512 for the new output size
LATENT_DIM = 200  # Latent dimension remains the same
BATCH_SIZE = 24  # Batch size remains the same
EPOCHS = 100

epoch_times = []  # List to store time taken for each epoch
d_loss_values = []
g_loss_values = []

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.init_size = IMG_SIZE // (2 ** 5)  # Adjusted for 512x512 images
        self.l1 = nn.Sequential(nn.Linear(LATENT_DIM, 256 * self.init_size ** 2))

        # Pretrained layers (up to 256x256 output)
        self.conv_blocks_pretrained = nn.Sequential(
            nn.BatchNorm2d(256),
            nn.Upsample(scale_factor=2),  # 16x16 -> 32x32
            nn.Conv2d(256, 256, 3, stride=1, padding=1),
            nn.BatchNorm2d(256, 0.8),
            nn.ReLU(inplace=True),

            nn.Upsample(scale_factor=2),  # 32x32 -> 64x64
            nn.Conv2d(256, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.ReLU(inplace=True),

            nn.Upsample(scale_factor=2),  # 64x64 -> 128x128
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.ReLU(inplace=True),

            nn.Upsample(scale_factor=2),  # 128x128 -> 256x256
            nn.Conv2d(64, 32, 3, stride=1, padding=1),
            nn.BatchNorm2d(32, 0.8),
            nn.ReLU(inplace=True),
        )

        # New layers to upscale from 256x256 to 512x512
        self.conv_blocks_new = nn.Sequential(
            nn.Upsample(scale_factor=2),  # 256x256 -> 512x512
            nn.Conv2d(32, 16, 3, stride=1, padding=1),
            nn.BatchNorm2d(16, 0.8),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, 3, 3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 256, self.init_size, self.init_size)
        out = self.conv_blocks_pretrained(out)
        img = self.conv_blocks_new(out)
        return img

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),  # 256x256 -> 128x128
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),  # 128x128 -> 64x64
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),  # 64x64 -> 32x32
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, 4, stride=2, padding=1),  # 32x32 -> 16x16
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1024, 4, stride=2, padding=1),  # 16x16 -> 8x8
            nn.BatchNorm2d(1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Flatten(),
            nn.Linear(1024 * 8 * 8, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        validity = self.model(img)
        return validity

transform = transforms.Compose([
    transforms.Resize(256),  # Keep the images at 256x256 for the Discriminator
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

dataloader = DataLoader(
    ConcatDataset([
        datasets.Flowers102(root='../../data/flowers', split='train', download=True, transform=transform),
        datasets.Flowers102(root='../../data/flowers', split='val', download=True, transform=transform),
        datasets.Flowers102(root='../../data/flowers', split='test', download=True, transform=transform)
    ]),
    batch_size=BATCH_SIZE, shuffle=True
)

# Initialize models
generator = Generator().to(device)
discriminator = Discriminator().to(device)

# Load pre-trained Generator weights
pretrained_dict = torch.load('saved_model/saved_model_dcgan_flower2_256_100.pth')
model_dict = generator.state_dict()

# Filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# Overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
# Load the new state dict
generator.load_state_dict(model_dict)

# Freeze pre-trained layers
for param in generator.l1.parameters():
    param.requires_grad = False

for param in generator.conv_blocks_pretrained.parameters():
    param.requires_grad = False

# Generate and save summary for Generator model
generator_summary = summary(generator, input_size=(1, LATENT_DIM), verbose=2, col_names=[
                            "input_size", "output_size", "num_params", "kernel_size", "mult_adds"])
with open("generator_summary_512_transfer.txt", "w", encoding="utf-8") as f:
    f.write(str(generator_summary))

# Generate and save summary for Discriminator model
discriminator_summary = summary(discriminator, input_size=(
    1, 3, 256, 256), verbose=2, col_names=["input_size", "output_size", "num_params", "kernel_size", "mult_adds"])
with open("discriminator_summary_256_transfer.txt", "w", encoding="utf-8") as f:
    f.write(str(discriminator_summary))

# Visualize the Generator Model
z_sample = torch.randn(1, LATENT_DIM).to(device)  # A single sample input for the generator
gen_sample_output = generator(z_sample)
gen_dot = make_dot(gen_sample_output, params=dict(generator.named_parameters()))
gen_dot.render("generator_architecture_512_transfer", format="png")  # Save the visualization as a PNG file

# Visualize the Discriminator Model
real_img_sample = torch.randn(1, 3, 256, 256).to(device)  # A single sample input for the discriminator
disc_sample_output = discriminator(real_img_sample)
disc_dot = make_dot(disc_sample_output, params=dict(discriminator.named_parameters()))
disc_dot.render("discriminator_architecture_256_transfer", format="png")  # Save the visualization as a PNG file

# Loss function
adversarial_loss = nn.BCELoss()

# Optimizers
optimizer_G = optim.Adam(filter(lambda p: p.requires_grad, generator.parameters()), lr=0.0002, betas=(0.5, 0.999))
optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))

# Training
for epoch in range(EPOCHS):
    epoch_start_time = time.time()  # Start time for the epoch
    for i, (imgs, _) in enumerate(dataloader):

        # Calculate and print GPU memory usage
        allocated_memory = torch.cuda.memory_allocated() / (1024 ** 3)  # Convert to GB
        reserved_memory = torch.cuda.memory_reserved() / (1024 ** 3)    # Convert to GB

        # Configure input
        real_imgs = imgs.to(device)
        batch_size = real_imgs.size(0)

        # Adversarial ground truths
        valid = torch.ones(batch_size, 1, requires_grad=False).to(device)
        fake = torch.zeros(batch_size, 1, requires_grad=False).to(device)

        # ---------------------
        #  Train Discriminator
        # ---------------------
        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = torch.randn(batch_size, LATENT_DIM).to(device)

        # Generate a batch of images
        gen_imgs = generator(z)

        # Downsample generated images to 256x256 for the Discriminator
        gen_imgs_resized = F.interpolate(gen_imgs, size=(256, 256), mode='bilinear', align_corners=False)

        # Loss for real images
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        # Loss for fake images
        fake_loss = adversarial_loss(discriminator(gen_imgs_resized.detach()), fake)
        # Total discriminator loss
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        # -----------------
        #  Train Generator
        # -----------------
        optimizer_G.zero_grad()

        # Loss for fake images with flipped labels
        g_loss = adversarial_loss(discriminator(gen_imgs_resized), valid)

        g_loss.backward()
        optimizer_G.step()

        print(f"[Epoch {epoch}/{EPOCHS}] [Batch {i}/{len(dataloader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")
        print(f"[GPU Memory Allocated: {allocated_memory:.2f} GB] [GPU Memory Reserved: {reserved_memory:.2f} GB]")

    # Save sample images
    if epoch % 2 == 0:
        save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_flowers2_512_transfer.png", nrow=5, normalize=True)
        # Save the model
        torch.save(generator.state_dict(), f"saved_model_dcgan_flower2_512_transfer_{epoch}.pth")
        d_loss_values.append(d_loss.item())
        g_loss_values.append(g_loss.item())
    epoch_end_time = time.time()  # End time for the epoch
    epoch_duration = epoch_end_time - epoch_start_time  # Duration of the epoch
    epoch_times.append(epoch_duration)  # Append the duration to the list

# Save model after last epoch
save_image(gen_imgs.data[:25], f"images/{epoch}_DCGAN_flowers2_512_transfer.png", nrow=5, normalize=True)
torch.save(generator.state_dict(), f"saved_model_dcgan_flower2_512_transfer_{EPOCHS}.pth")

# Calculate average time per epoch
average_time_per_epoch = sum(epoch_times) / len(epoch_times)
print(f"Average time per epoch: {average_time_per_epoch:.2f} seconds")

# Plot loss values
plt.plot(np.arange(0, EPOCHS, 2), d_loss_values, label='Discriminator loss')
plt.plot(np.arange(0, EPOCHS, 2), g_loss_values, label='Generator loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss values')
plt.savefig('loss_values_flowers_512_transfer.png')
plt.show()

# Plot time taken per epoch
plt.plot(np.arange(0, EPOCHS), epoch_times)
plt.title("Time taken per epoch")
plt.xlabel("Epoch")
plt.ylabel("Time (s)")
plt.savefig("time_per_epoch_flowers_512_transfer.png")
plt.show()


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
Generator                                [1, 200]                  [1, 3, 512, 512]          --                        --                        --
├─Sequential: 1-1                        [1, 200]                  [1, 65536]                --                        --                        --
│    └─0.weight                                                                              ├─13,107,200              [65536, 200]
│    └─0.bias                                                                                └─65,536                  [65536]
│    └─Linear: 2-1                       [1, 200]                  [1, 65536]                (13,172,736)              --                        13,172,736
│    │    └─weight                                                                           ├─13,107,200              [200, 65536]
│ 